Features of the Phonebook Application
Linked List Data Structure:

Contacts are stored using a linked list, allowing efficient addition and removal of contacts.
Contact Management:

Add Contact: Users can add new contacts by providing a name and a number.
Remove Contact: Users can remove contacts by searching for them using either the name or the number.
Search Contact: Users can search for contacts by entering a partial name, which shows a list of matching contacts. This provides functionality similar to autocomplete.
Duplicate Name Handling:

The application checks for existing contacts with the same name before adding a new one.
If a duplicate is found, the user is prompted to either update the existing contact or create a new one with a modified name (e.g., "Sumit" becomes "Sumit2").
Persistence:

Contacts are saved to a JSON file (contacts.json) when the application closes, ensuring that data is not lost between sessions.
The application can load existing contacts from the JSON file upon startup.
Graphical User Interface (GUI):

Built using the tkinter library, the GUI allows users to interact with the phonebook easily.
Input fields for adding and searching contacts, as well as buttons for adding, removing, and displaying contacts.
Display of Contacts:

A text area in the GUI shows all contacts, allowing users to view their phonebook at a glance.
The contact display updates dynamically as contacts are added or removed.
User Notifications:

The application uses message boxes to notify users about successful additions, updates, removals, and warnings for incorrect inputs.
Efficient Data Handling:

The linked list allows for efficient addition and removal of contacts, while searching for contacts is optimized using linear search methods.

In [7]:
import tkinter as tk
from tkinter import messagebox
import json

class Contact:
    def __init__(self, name, number):
        self.name = name
        self.number = number
        self.next = None

class Phonebook:
    def __init__(self):
        self.head = None

    # Add contact to the linked list
    def add_contact(self, name, number):
        existing_contact = self.search_by_name(name)
        if existing_contact:
            # If contact exists, ask user what to do
            response = messagebox.askyesno(
                "Contact Exists",
                f"Contact with name '{name}' already exists.\n"
                "Do you want to save this number under the existing contact?"
            )
            if response:
                existing_contact.number = number  
                return
            
            # If user wants to create a new contact, modify the name
            suffix = 2
            new_name = f"{name}{suffix}"
            while self.search_by_name(new_name):
                suffix += 1
                new_name = f"{name}{suffix}"
            name = new_name

        # Create a new contact
        new_contact = Contact(name, number)
        if self.head is None:
            self.head = new_contact
        else:
            current = self.head
            while current.next:
                current = current.next
            current.next = new_contact

    # Remove contact from the linked list 
    def remove_contact(self, query):
        if self.head is None:
            print("Phonebook is empty.")
            return

        # If the contact to delete is the head
        if self.head.name == query or self.head.number == query:
            self.head = self.head.next
            return

        current = self.head
        while current.next and (current.next.name != query and current.next.number != query):
            current = current.next

        if current.next:
            current.next = current.next.next
        else:
            print(f"No contact found with name or number: {query}")

    # Search contact by name 
    def search_by_name(self, name):
        current = self.head
        while current:
            if current.name.lower() == name.lower():  # Case-insensitive search
                return current
            current = current.next
        return None

    # Search contact
    def search_by_partial_name(self, partial_name):
        results = []
        current = self.head
        while current:
            if partial_name.lower() in current.name.lower():  # Case-insensitive search
                results.append(current)
            current = current.next
        return results

    # Display all contacts as a list of tuples
    def get_all_contacts(self):
        contacts = []
        current = self.head
        while current:
            contacts.append((current.name, current.number))
            current = current.next
        return contacts

# Save and Load from File
def save_contacts(phonebook, filename="contacts.json"):
    try:
        contacts = []
        current = phonebook.head
        while current:
            contacts.append({"name": current.name, "number": current.number})
            current = current.next

        with open(filename, 'w') as file:
            json.dump(contacts, file)
    except Exception as e:
        print(f"Error saving contacts: {e}")

def load_contacts(phonebook, filename="contacts.json"):
    try:
        with open(filename, 'r') as file:
            contacts = json.load(file)

        for contact in contacts:
            phonebook.add_contact(contact['name'], contact['number'])

    except FileNotFoundError:
        print(f"{filename} not found. Starting with an empty phonebook.")

# Using GUI - tkinter
class PhonebookGUI:
    def __init__(self, phonebook):
        self.phonebook = phonebook
        self.root = tk.Tk()
        self.root.title("Phonebook")
        
        # Inputs
        self.name_label = tk.Label(self.root, text="Name")
        self.name_label.pack()
        self.name_entry = tk.Entry(self.root)
        self.name_entry.pack()

        self.number_label = tk.Label(self.root, text="Number")
        self.number_label.pack()
        self.number_entry = tk.Entry(self.root)
        self.number_entry.pack()

        # Buttons
        self.add_button = tk.Button(self.root, text="Add Contact", command=self.add_contact)
        self.add_button.pack()

        self.search_button = tk.Button(self.root, text="Search Contact", command=self.search_contact)
        self.search_button.pack()

        self.remove_button = tk.Button(self.root, text="Remove Contact", command=self.remove_contact)
        self.remove_button.pack()

        self.display_button = tk.Button(self.root, text="Display Contacts", command=self.display_contacts)
        self.display_button.pack()

        # Display contacts
        self.contacts_text = tk.Text(self.root, height=10, width=50)
        self.contacts_text.pack()

    def add_contact(self):
        name = self.name_entry.get()
        number = self.number_entry.get()
        if name and number:
            self.phonebook.add_contact(name, number)
            messagebox.showinfo("Success", f"Contact {name} added or updated.")
            self.display_contacts()
        else:
            messagebox.showwarning("Input Error", "Please enter both name and number.")

    def search_contact(self):
        partial_name = self.name_entry.get()
        results = self.phonebook.search_by_partial_name(partial_name)
        self.contacts_text.delete(1.0, tk.END)  
        if results:
            self.contacts_text.insert(tk.END, "Search results:\n")
            for contact in results:
                self.contacts_text.insert(tk.END, f"Name: {contact.name}, Number: {contact.number}\n")
        else:
            self.contacts_text.insert(tk.END, "No matching contacts found.\n")

    def remove_contact(self):
        query = self.name_entry.get() or self.number_entry.get()  
        if query:
            self.phonebook.remove_contact(query)
            messagebox.showinfo("Success", f"Contact with name or number {query} removed.")
            self.display_contacts()
        else:
            messagebox.showwarning("Input Error", "Please enter a name or number to remove.")

    def display_contacts(self):
        contacts = self.phonebook.get_all_contacts()
        self.contacts_text.delete(1.0, tk.END)
        if contacts:
            for name, number in contacts:
                self.contacts_text.insert(tk.END, f"Name: {name}, Number: {number}\n")
        else:
            self.contacts_text.insert(tk.END, "No contacts available.\n")

    def run(self):
        self.root.protocol("WM_DELETE_WINDOW", self.on_closing) 
        self.root.mainloop()

    def on_closing(self):
        save_contacts(self.phonebook) 
        self.root.destroy()

# Initialize the phonebook and GUI
phonebook = Phonebook()
load_contacts(phonebook)

app = PhonebookGUI(phonebook)
app.run()
